In [176]:
import pandas as pd
import numpy as np 
import re

In [177]:
df = pd.read_csv(r'movies_profi.csv')

In [178]:
df.head()

,Month,Day,Title,Link,List,Year,Genres,Budget,Box office
0,1,2.0,The Woman in Black: Angel of Death,/wiki/The_Woman_in_Black:_Angel_of_Death,"['supernatural horror', '$15 million[6]', '$48...",2015.0,supernatural horror,$15 million[6],$48.9 million[1]
1,1,9.0,Taken 3,/wiki/Taken_3,"['-language French action-thriller', '$48 mill...",2015.0,-language French action-thriller,$48 million[2],$326.4 million[2]
2,1,16.0,Blackhat,/wiki/Blackhat_(film),"[' action thriller', '$70 million[3]', '$19.7 ...",2015.0,action thriller,$70 million[3],$19.7 million[3]
3,1,16.0,Spare Parts,/wiki/Spare_Parts_(2015_film),"['drama', '$4 million[2]', '$3.6 million[3]']",2015.0,drama,$4 million[2],$3.6 million[3]
4,1,16.0,The Wedding Ringer,/wiki/The_Wedding_Ringer,"[' buddy romantic comedy', '$23 million[4]', '...",2015.0,buddy romantic comedy,$23 million[4],$79.8 million[4]


In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Month       1032 non-null   int64  
 1   Day         1032 non-null   float64
 2   Title       1032 non-null   object 
 3   Link        1032 non-null   object 
 4   List        1032 non-null   object 
 5   Year        1032 non-null   float64
 6   Genres      1032 non-null   object 
 7   Budget      1032 non-null   object 
 8   Box office  1032 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 72.7+ KB


### Cleaning the Budget col

In [180]:
df['Budget'] = df['Budget'].str.split('[').str.get(0)
df['Box office'] = df['Box office'].str.split('[').str.get(0)
df.tail()

,Month,Day,Title,Link,List,Year,Genres,Budget,Box office
1027,12,16.0,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,"[' epic science fiction', '$350–460\xa0million...",2022.0,epic science fiction,$350–460 million,$2.32 billion
1028,12,21.0,Puss in Boots: The Last Wish,/wiki/Puss_in_Boots:_The_Last_Wish,"[' animated adventure comedy', '$90–110 millio...",2022.0,animated adventure comedy,$90–110 million,$484.6 million
1029,12,23.0,Babylon,/wiki/Babylon_(2022_film),"[' epic period black comedy-drama', '$78–80 mi...",2022.0,epic period black comedy-drama,$78–80 million,$63.4 million
1030,12,23.0,Whitney Houston: I Wanna Dance with Somebody,/wiki/Whitney_Houston:_I_Wanna_Dance_with_Some...,"[' biographical musical drama', '$45 million[1...",2022.0,biographical musical drama,$45 million,$59.8 million
1031,12,29.0,A Man Called Otto,/wiki/A_Man_Called_Otto,"[' comedy-drama', '$50 million', '$113.2 milli...",2022.0,comedy-drama,$50 million,$113.2 million


In [181]:
df.rename(columns={'Budget':'Budget2'}, inplace=True)

In [182]:
df['Budget2'] = df['Budget2'].str.replace(r'–', '-')

In [183]:
mask = df['Budget2'].str.find('-')
mask = mask != -1

In [184]:
(df.loc[mask, 'Budget2'].str.find('million') == -1).any()

False

In [185]:
def get_mean_budget(x):
    pattern = r'\$?([\d\.]*)-\$?([\d\.]*)'
    try:
        finds = re.findall(pattern, x)[0]
        return (float(finds[0]) + float(finds[1])) / 2
    except:
        pattern = r'\$?([\d\.]*)[\s\w]*-[\s\$]?([\d\.]*)'
        finds = re.findall(pattern, x)[0]
        return (float(finds[0]) + float(finds[1])) / 2      


df['Budget'] = df.loc[mask, 'Budget2'].apply(get_mean_budget)

In [186]:
mask = df['Budget'].isna()

In [187]:
(df.loc[mask, 'Budget2'].str.find('million') == -1).sum()

19

In [188]:
mask2 = (df.loc[mask, 'Budget2'].str.find('million') == -1)
df[mask].loc[mask2, 'Budget2']

82                 $100,000
84                 $100,000
200                $900,000
343                $500,000
356                 $80,000
402                $900,000
413                $880,000
659     $66 thousand (est.)
666                $586,443
671                £850,000
730                $500,000
734                $200,000
788               <$300,000
797                $250,000
871                $250,000
885                $950,000
902                $100,000
988                $500,000
1003               $250,000
Name: Budget2, dtype: object

In [189]:
df.loc[659, 'Budget'] = 0.066

In [190]:
def thousands_budget(x):
    pattern = r'.?\$(\d+),'
    try:
        finds = re.findall(pattern, x)[0]
        return float(finds) / 1000
    except:
        return 0.066
    
df.loc[(mask & mask2), 'Budget'] = df[mask].loc[mask2, 'Budget2'].apply(thousands_budget)

In [191]:
df.tail()

,Month,Day,Title,Link,List,Year,Genres,Budget2,Box office,Budget
1027,12,16.0,Avatar: The Way of Water,/wiki/Avatar:_The_Way_of_Water,"[' epic science fiction', '$350–460\xa0million...",2022.0,epic science fiction,$350-460 million,$2.32 billion,405.0
1028,12,21.0,Puss in Boots: The Last Wish,/wiki/Puss_in_Boots:_The_Last_Wish,"[' animated adventure comedy', '$90–110 millio...",2022.0,animated adventure comedy,$90-110 million,$484.6 million,100.0
1029,12,23.0,Babylon,/wiki/Babylon_(2022_film),"[' epic period black comedy-drama', '$78–80 mi...",2022.0,epic period black comedy-drama,$78-80 million,$63.4 million,79.0
1030,12,23.0,Whitney Houston: I Wanna Dance with Somebody,/wiki/Whitney_Houston:_I_Wanna_Dance_with_Some...,"[' biographical musical drama', '$45 million[1...",2022.0,biographical musical drama,$45 million,$59.8 million,NaN
1031,12,29.0,A Man Called Otto,/wiki/A_Man_Called_Otto,"[' comedy-drama', '$50 million', '$113.2 milli...",2022.0,comedy-drama,$50 million,$113.2 million,NaN


In [192]:
mask2 = (df.loc[mask, 'Budget2'].str.find('million') != -1)
df[mask].loc[mask2, 'Budget2'][:5]

0    $15 million
1    $48 million
2    $70 million
3     $4 million
4    $23 million
Name: Budget2, dtype: object

In [193]:
def millions_budget(x):
    pattern = r'[\$€£](\d+)'
    finds = re.findall(pattern, x)
    if len(finds) == 0:
        print('Error' + x)
    else:
        return float(finds[0])

df.loc[(mask & mask2), 'Budget'] = df[mask].loc[mask2, 'Budget2'].apply(millions_budget)

In [194]:
df.head()

,Month,Day,Title,Link,List,Year,Genres,Budget2,Box office,Budget
0,1,2.0,The Woman in Black: Angel of Death,/wiki/The_Woman_in_Black:_Angel_of_Death,"['supernatural horror', '$15 million[6]', '$48...",2015.0,supernatural horror,$15 million,$48.9 million,15.0
1,1,9.0,Taken 3,/wiki/Taken_3,"['-language French action-thriller', '$48 mill...",2015.0,-language French action-thriller,$48 million,$326.4 million,48.0
2,1,16.0,Blackhat,/wiki/Blackhat_(film),"[' action thriller', '$70 million[3]', '$19.7 ...",2015.0,action thriller,$70 million,$19.7 million,70.0
3,1,16.0,Spare Parts,/wiki/Spare_Parts_(2015_film),"['drama', '$4 million[2]', '$3.6 million[3]']",2015.0,drama,$4 million,$3.6 million,4.0
4,1,16.0,The Wedding Ringer,/wiki/The_Wedding_Ringer,"[' buddy romantic comedy', '$23 million[4]', '...",2015.0,buddy romantic comedy,$23 million,$79.8 million,23.0


In [196]:
df['Budget'].isna().sum()

0